In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
# We won't need TensorFlow here
%pip uninstall -y tensorflow
# Install `transformers` from master
%pip install git+https://github.com/huggingface/transformers
%pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1

Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-4k0jo8gf
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-4k0jo8gf
  Resolved https://github.com/huggingface/transformers to commit 2c658b5a4282f2e824b4e23dc3bcda7ef27d5827
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8119970 sha256=54373ca41cda4bb06bd0226ea1232845e3cf2620a7492a28f91d9954f0faa129
  Stored in directory: /tmp/pip-ephem-wheel-cache-t3k0r58z/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninst

In [3]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "composer-classification/note-classifier-vocab.json",
    "composer-classification/note-classifier-merges.txt",
)

In [4]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [5]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=50_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

/home/jipe/anaconda3/envs/E208/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast("composer-classification/note-classifier-vocab.json", "composer-classification/note-classifier-merges.txt", max_len=512)

In [11]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [12]:
model.num_parameters()
# => 84 million parameters

81966416

In [13]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="sentences.txt",
    block_size=128,
)

/home/jipe/anaconda3/envs/E208/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


CPU times: user 1min 41s, sys: 1.18 s, total: 1min 43s
Wall time: 5.14 s


In [14]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [18]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="composer-classification",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [19]:
%%time
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/jipe/anaconda3/envs/E208/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss


CPU times: user 4min 11s, sys: 2.25 s, total: 4min 13s
Wall time: 2min 27s


TrainOutput(global_step=313, training_loss=4.585713249425918, metrics={'train_runtime': 147.479, 'train_samples_per_second': 271.259, 'train_steps_per_second': 2.122, 'total_flos': 1326375701360640.0, 'train_loss': 4.585713249425918, 'epoch': 1.0})

In [20]:
trainer.save_model("composer-classification")

In [ ]:
# from tokenizers.implementations import ByteLevelBPETokenizer
# from tokenizers.processors import BertProcessing
# from torch.utils.data import Dataset

# class ComposerClassificationDataset(Dataset):
#     def __init__(self, evaluate: bool = False):
#         tokenizer = ByteLevelBPETokenizer(
#             "note-classifier-vocab.json",
#             "note-classifier-merges.txt",
#         )
#         tokenizer._tokenizer.post_processor = BertProcessing(
#             ("</s>", tokenizer.token_to_id("</s>")),
#             ("<s>", tokenizer.token_to_id("<s>")),
#         )
#         tokenizer.enable_truncation(max_length=512)
#         # or use the RobertaTokenizer from `transformers` directly.

#         self.examples = []

#         src_files = 'sentences.txt'
#         for src_file in src_files:
#             print("🔥", src_file)
#             lines = src_file.read_text(encoding="utf-8").splitlines()
#             self.examples += [x.ids for x in tokenizer.encode_batch(lines)]

#     def __len__(self):
#         return len(self.examples)

#     def __getitem__(self, i):
#         # We’ll pad at the batch level.
#         return torch.tensor(self.examples[i])